# 필요한 라이브러리 임포트

In [1]:
import chardet
import csv
import os
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# 크롬 드라이버 설정

In [2]:
# 크롬드라이버 서비스를 설정합니다.
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)

# 크롬 드라이버 열기

In [3]:
# 대한민국 대통령 연설문 페이지에 접속합니다.
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')

# 웹페이지 내부 크롤링

In [ ]:
# 특수 문자를 제외한 모든 문자를 유효한 문자로 변환하는 테이블 생성
valid_chars = {ord(c): "_" for c in f'{"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_ "}'}

def get_unique_file_name(file_name, output_dir):
    index = 1
    unique_file_name = file_name
    while os.path.exists(os.path.join(output_dir, unique_file_name)):
        name, ext = os.path.splitext(file_name)
        unique_file_name = f"{name}_{index}{ext}"
        index += 1
    return unique_file_name

# 무한 루프를 생성하여 웹 페이지를 계속해서 크롤링
while True:
    # 각 페이지의 메뉴 항목을 반복 처리
    for i in range(3, 14):
        xpath = f"/html/body/main/div/form[1]/nav[2]/ul/li[{i}]"
        element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        element.click()
        elements1 = driver.find_elements(By.CSS_SELECTOR, '#M_More > tr')
        time.sleep(1)

        # 웹 페이지의 소스를 가져옴
        soup1 = BeautifulSoup(driver.page_source, 'html.parser')

        # 각 연설문 항목을 반복 처리
        for num_elements1 in range(len(elements1)):
            
            xpath1 = f"/html/body/main/div/form[1]/table/tbody/tr[{num_elements1 + 1}]/td[5]/a"
            soup1 = BeautifulSoup(driver.page_source, 'html.parser')
            speech_num = soup1.select_one(f'#M_More > tr:nth-child({num_elements1 + 1}) > td:nth-child(1)')
            
            # 각 연설문 항목의 xpath를 생성
            element1 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath1)))
            element1.click()
            
            elements2 = driver.find_elements(By.XPATH, '/html/body/div[2]/div[1]/div/div[2]/div/ul/li')
            
            soup2 = BeautifulSoup(driver.page_source, 'html.parser')
            speech_num = soup1.select_one(f'#M_More > tr:nth-child({num_elements1 + 1}) > td:nth-child(1)')
            speech_person = soup1.select_one(f'#M_More > tr:nth-child({num_elements1 + 1}) > td:nth-child(2) > a')
            speech_form = soup1.select_one(f'#M_More > tr:nth-child({num_elements1 + 1}) > td:nth-child(4) > a')
            speech_title = soup1.select_one(f'#M_More > tr:nth-child({num_elements1 + 1}) > td:nth-child(5) > a')
            speech_days = soup1.select_one(f'#M_More > tr:nth-child({num_elements1 + 1}) > td:nth-child(6)')
            content_tag = soup2.select_one('#content > div > table > tbody > tr:nth-child(4) > td')
 
            # 파일 이름 생성
                    # 파일 이름 생성
            file_title = speech_title.text.strip().replace('/', '-').replace('\\', '').replace('\'', '').replace('\"', '').replace('<', '').replace('>', '').replace(':', '').replace('*', '').replace('?', '').replace('|', '').replace('\n', '').replace('\r', '')
            file_name = f'{file_title}.csv'  # 파일 이름을 생성합니다.

            # 파일 경로 생성
            output_dir = "Every President's Speech"  # 파일이 저장될 디렉터리 이름
            if not os.path.exists(output_dir):  # 디렉터리가 존재하지 않으면 생성
                os.makedirs(output_dir)

            # 중복되지 않는 파일 이름 생성
            unique_file_name = get_unique_file_name(file_name, output_dir)
            file_path = os.path.join(output_dir, unique_file_name)

            # CSV 파일로 저장
            with open(file_path, 'w', encoding='utf-8', newline='') as f:
                csv_writer = csv.writer(f)
                csv_writer.writerow([speech_num, speech_title, speech_person, speech_form, speech_days, content_tag.text])

            # 카테고리로 돌아가기 위해 클릭
            try:
                category = driver.find_element(By.XPATH, '/html/body/main/div/div/div/a[2]')
            except NoSuchElementException:
                category = driver.find_element(By.XPATH, '/html/body/main/div/div/div/a')

            category.click()

        # 조건에 해당하는 연설문이 발견되면 루프 종료
        if file_title == "조계종 제15대 종정 추대 법회":
            break

    # 조건에 해당하는 연설문이 발견되면 루프 종료
    if file_title == "조계종 제15대 종정 추대 법회":
        break


# 줄바꿈 제거

In [ ]:
# CSV 파일이 있는 최상위 디렉토리 경로
directory = r"C:\Users\Admin\Desktop\for_code\Every President's Speech3"

# 디렉토리와 하위 폴더에서 모든 CSV 파일을 가져와서 처리
for root, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".csv"):
            file_path = os.path.join(root, filename)
            # CSV 파일 열기
            with open(file_path, 'rb') as f:
                # 파일 내용 읽어오기
                content = f.read()
                # 파일 인코딩 감지
                encoding = chardet.detect(content)['encoding']
            # 파일 읽기 (인코딩 설정 추가)
            with open(file_path, 'r', encoding=encoding) as f:
                # 파일 내용 읽어오기
                content = f.read()
            # 파일 내용의 개행 문자와 탭 문자 제거
            content = content.replace('\n', '').replace('\t', '').replace('Wr', '')
            # CSV 파일 다시 쓰기
            with open(file_path, 'w', encoding=encoding, newline='') as f:
                f.write(content)

# 엑셀 파일 누락값 확인기

In [ ]:
import pandas as pd

# 저장된 CSV 파일 경로를 입력하세요.
file_path = "/Users/Admin/Desktop/전체 대통령 파일2.csv"

# CSV 파일을 읽어옵니다.
data = pd.read_csv(file_path)

# 1번 컬럼의 값들을 가져옵니다.
column_values = data.iloc[:, 0].values

# 1부터 8565까지의 연속된 값들을 생성합니다.
expected_values = set(range(1, 8566))

# 실제 컬럼 값들과 비교하여 빠져 있는 값들을 찾습니다.
missing_values = expected_values - set(column_values)

# 결과를 출력합니다.
if missing_values:
    print(f"다음 값들이 빠져 있습니다: {sorted(missing_values)}")
else:
    print("1번 컬럼은 1부터 8565까지 끊기는 부분 없이 이어집니다.")